### Bài 1

In [321]:
from bs4 import BeautifulSoup
import pandas as pd

#### 1. Hiển thị tất cả các sinh viên

In [322]:
df_students = pd.read_xml('../Lab3/data/students.xml')
df_students

,id,name,age,city
0,1,Becka,19,HCM
1,10,Anna,27,NT
2,2,Ray,18,HCM
3,3,Rayme,20,DN
4,4,Jack,18,HN
5,5,Kevin,30,CT
6,6,Rocky,21,DN
7,7,Elsa,18,HN
8,8,Jackli,22,HCM
9,9,Alson,23,HCM


#### 2. Hiển thị danh sách sinh viên theo city

In [323]:
def find_by_city(df, city_name):

    if not city_name: # if city_name == None, [], false, ...
        print("Lỗi nhập dữ liệu")
        return 

    rls = df[df['city'] == city_name]

    # Nếu df là không thì hiển thị thông báo và sau đó là thoát khỏi hàm
    if rls.empty:
        print("Không tìm thấy dữ liệu sinh viên")
        return 
        
    # trả về df nếu tìm thấy dữ liệu
    else:
        return rls.sort_values(by = 'id').reset_index(drop=True)

#### 3. Tìm sinh viên theo tên

In [324]:
def find_by_student_name(df, student_name:str):

    if not student_name:
        print("Lỗi nhập dữ liệu")
        return 
    
    rls = df[df['name'] == student_name]

    if rls.empty:
        print("Không tìm thấy dữ liệu sinh viên")
        return
        
    else:
        return rls.sort_values(by = 'id').reset_index(drop=True)

#### 4. Sắp xếp danh sách sinh viên theo tuổi và ghi vào file xml mới theo thứ tự đã sắp xếp

In [325]:
def export_and_record_xml(df) -> None:
    # Đảm bảo kiểu dữ liệu chuẩn để tránh lỗi khi ghi XML
    df['id'] = df['id'].astype(str)
    df['age'] = pd.to_numeric(df['age'], errors='coerce')

    # Xuất ra XML
    new_xml = df.to_xml(root_name='students', row_name='student', index=False)

    # Ghi vào file
    with open('../Lab3/data/students.xml', 'w', encoding='utf-8') as f:
        f.write(new_xml)


In [326]:
# Mặc định sẽ là sắp xếp tăng dần
def sortBy_age(df, ascending = True):

    # Sắp xếp theo tuổi, mặc định là tăng dần
    rls = df.sort_values(by = 'age', ascending=ascending)

    new_xml = rls.to_xml(root_name = 'students', row_name = 'student', index =False)
    with open('new_students.xml', 'w') as f:
        f.write(new_xml)

    return 

#### 5. Thêm sinh viên mới và ghi lại file

In [327]:
def insert_student_to_XML(xml_file, input_id, input_name, input_age, input_city):
    import pandas as pd

    # Đọc file XML
    df_students = pd.read_xml(xml_file)

    # Tạo DataFrame mới từ dữ liệu đầu vào
    new_student = pd.DataFrame([{
        'id': str(input_id),
        'name': input_name,
        'age': int(input_age),
        'city': input_city
    }])

    # Ép kiểu id về chuỗi để tránh lỗi sort
    df_students['id'] = df_students['id'].astype(str)

    # Nối và sắp xếp
    df_students = pd.concat([df_students, new_student], ignore_index=True)
    df_students = df_students.sort_values(by='id')

    # Ghi đè lại file XML
    export_and_record_xml(df_students)

    return None


#### 6. Sửa dữ liệu của sinh viên theo ID

In [328]:
# Điều kiện để thực hiện xoá hoặc sửa dữ liệu
def findBy_id(df: pd.DataFrame, input_id: str) -> bool:
    df['id'] = df['id'].astype(str)
    return not df[df['id'] == input_id].empty

    

def edit_infor(df: pd.DataFrame, input_id: str, new_name: str, new_age: str, new_city: str) -> None:
    mask = df['id'] == input_id

    # Cập nhật
    if new_name:
        df.loc[mask, 'name'] = new_name

    if new_age:
        try:
            df.loc[mask, 'age'] = int(new_age)
        except ValueError:
            print("Tuổi phải là số.")

    if new_city:
        df.loc[mask, 'city'] = new_city

    export_and_record_xml(df)
    return None


#### 7. Xoá sinh viên và ghi lại vào file

In [329]:
def del_student(df: pd.DataFrame, input_id: str):
    df['id'] = df['id'].astype(str)
    student_mask = df['id'] == input_id

    df.drop(index=df[student_mask].index, inplace=True)

    # Ghi lại file XML
    export_and_record_xml(df)
    return df


### Menu

In [ ]:
def menu():
    # Đọc file XML
    df_students = pd.read_xml('../Lab3/data/students.xml')
    
    while True:
        print("\n--- MENU ---")
        print("1. Hiển thị tất cả các sinh viên")
        print("2. Hiển thị danh sách sinh viên theo city")
        print("3. Tìm sinh viên theo tên")
        print("4. Sắp xếp danh sách sinh viên theo tuổi và ghi vào file xml mới")
        print("5. Thêm sinh viên mới và ghi lại file")
        print("6. Sửa dữ liệu của sinh viên theo ID")
        print("7. Xoá sinh viên và ghi lại vào file")
        print("0. Thoát")
        
        choice = input("Nhập lựa chọn của bạn: ")
        
        if choice == "1":
            print(df_students)
        
        elif choice == "2":
            city_name = input("Nhập tên city: ")
            result = find_by_city(df_students, city_name)
            if result is not None:
                print(result)
        
        elif choice == "3":
            student_name = input("Nhập tên sinh viên: ")
            result = find_by_student_name(df_students, student_name)
            if result is not None:
                print(result)
        
        elif choice == "4":
            ascending = input("Sắp xếp tăng dần? (y/n): ").lower()

            if ascending == 'y':
                arg = True
            else:
                arg = False

            sortBy_age(df_students, ascending=arg) 
            print("Dữ liệu đã được sắp xếp và xuất ra file XML:")
        
        elif choice == "5":
            input_id = input("Nhập ID: ")
            input_name = input("Nhập tên: ")
            input_age  = input("Nhập tuổi: ")
            input_city = input("Nhập city: ")
            try:
                insert_student_to_XML('../Lab3/data/students.xml', input_id, input_name, int(input_age), input_city)
                print("Thêm sinh viên thành công!")
            except Exception as e:
                print(f"Lỗi: {e}")
        
        elif choice == "6":
            input_id = input("Nhập ID của sinh viên muốn sửa: ")
            if findBy_id(df_students, input_id):
                new_name = input("Nhập tên mới: ")
                new_age = input("Nhập tuổi mới: ")
                new_city = input("Nhập city mới: ")
                edit_infor(df_students, input_id, new_name, new_age, new_city)
                print("Đã sửa thành công.")
            else:
                print("Không tìm thấy sinh viên với ID đã nhập.")
        
        elif choice == "7":
            input_id = input("Nhập ID của sinh viên muốn xóa: ").strip()
            if not findBy_id(df_students, input_id):
                print("Không tìm thấy sinh viên với ID đã nhập.")
                continue

            df_after_delete = del_student(df_students, input_id)
            print("Đã xoá thành công. Dữ liệu hiện tại:")
            print(df_after_delete)

            
        
        elif choice == "0":
            print("Thoát chương trình.")
            break
        
        else:
            print("Lựa chọn không hợp lệ. Vui lòng thử lại.")

# Gọi hàm menu
menu()


--- MENU ---
1. Hiển thị tất cả các sinh viên
2. Hiển thị danh sách sinh viên theo city
3. Tìm sinh viên theo tên
4. Sắp xếp danh sách sinh viên theo tuổi và ghi vào file xml mới
5. Thêm sinh viên mới và ghi lại file
6. Sửa dữ liệu của sinh viên theo ID
7. Xoá sinh viên và ghi lại vào file
0. Thoát

--- MENU ---
1. Hiển thị tất cả các sinh viên
2. Hiển thị danh sách sinh viên theo city
3. Tìm sinh viên theo tên
4. Sắp xếp danh sách sinh viên theo tuổi và ghi vào file xml mới
5. Thêm sinh viên mới và ghi lại file
6. Sửa dữ liệu của sinh viên theo ID
7. Xoá sinh viên và ghi lại vào file
0. Thoát
Thoát chương trình.


## Lỗi 5,6,7